# Delivery Duration Prediction
###### This is a machine learning project made with Doordash data. It is possible to find it at the following link.

# Importng Libraries

In [1]:
import pandas as pd #Dataframe creation and manipulation (any operation on data)
import numpy as np #Numerical mathematical operation
import matplotlib # Visualization
import matplotlib.pyplot as plt
import seaborn as sns # Visualization
%matplotlib inline # To visualize the plots in jupyter notebooks

sns.set_style("darkgrid") 
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (15, 5)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

import warnings
warnings.simplefilter(action='ignore')

UsageError: unrecognized arguments: # To visualize the plots in jupyter notebooks


In [ ]:
dd = pd.read_csv('historical_data.csv', parse_dates=['created_at', 'actual_delivery_time']) #to parse the column in date time format 
# parse_dates function of pandas to define the range from creayed_At to actual_delivery_time
dd

# EDA and Feature Engineering

In [ ]:
dd.info()

In [ ]:
dd.describe()
# mean, median- when mean is near to the 75% quartile data then mean is used
# subtotal mean lies between 505 and 75% quartile i.e data is right skewed thus we will choose mean for null value replacement, most of our data lies after 50%
# mode- If 25, 50 and 75% quartile data are nearly same then that means the most frequency of the data lies in one freq called mode
# max value can be a outlier
# If the data has less variance then we use mode for null value treatment

In [ ]:
dd.isna().sum()

In [ ]:
# Plotting null values using heatmap
plt.figure(figsize=(30,5))
sns.heatmap(dd.isna())
plt.xticks(rotation=90);

###### Since there are only seven missing values for one of the feature that will compose the target, it is possible to drop them.

In [ ]:
obs_to_remove = dd[dd.actual_delivery_time.isna()].index
dd.drop(index=obs_to_remove, inplace= True) # inplace is used to perform operation on existing dataframe without assigning to the variable
# To assign the changes to the dataset

In [ ]:
# Again counting the number of values after dropping
dd.actual_delivery_time.isna().sum()

# Creating the target column

In [ ]:
dd['Target'] = (dd.actual_delivery_time - dd.created_at).dt.seconds
dd.Target

# Replacing Missing Values

###### Columns with missing values:

* market_id
* store_primary_category
* order protocol
* total_onshift_dashers
* total_busy_dashers
* total_outstanding_orders
* estimated_store_to_consumer_driving_duration

# Market ID

In [ ]:
dd.market_id.value_counts() # Counts of particular market id, count the values in each category

###### ID 2.0 seems to be the most popular one, since there are not many information regarding the geographical position a possible solution is to replace the null values with 2.0

In [ ]:
dd.market_id.fillna(2.0, inplace=True)
print('Null values: {}'.format(dd.market_id.isna().sum()))
# print(f'Null values: {dd.market_id.isna().sum()}')
dd.market_id.value_counts()

# Store primary category

In [ ]:
dd.store_primary_category.isna().sum()

In [ ]:
'''unique_id = dd.store_id.unique()
unique_id
unique_id = dd.store_id.unique().tolist()# Convert array to list'''

In [ ]:
# new_df = pd.DataFrame() # create blank dataframe
dd[dd.store_id == 1845]['store_primary_category'].mode()[0]
# For store_id 1845 we found the mode of column store_primary_category to replce the null value

###### The missing values in the store primary category can be replaced by using the store_id feature.

In [ ]:
unique_id = dd.store_id.unique().tolist()
new_df = pd.DataFrame()

for x in unique_id:
    try:
        pop = dd[dd.store_id == x]['store_primary_category'].mode()[0]
        store_df = dd[dd.store_id == x]['store_primary_category'].fillna(pop)
        new_df = pd.concat([new_df, store_df])
    except:
        df_to_concat = dd[dd.store_id == x]['store_primary_category']
        new_df = pd.concat([new_df, df_to_concat])

In [ ]:
new_df

In [ ]:
new_df[0].isna().sum() 

In [ ]:
dd['store_primary_category'] = new_df[0] # Store the new column 0 to store_primary_category
dd.head()

###### The number of missing values dropped from 4760 to 867, in order to replace the remaining ones it could be adopted the same process but using the location feature.

In [ ]:
market_dict = {}

In [ ]:
def market_sub(id):
    popular = dd[dd.market_id == id]['store_primary_category'].mode()[0] # Here we are finding the mode for id
    market_dict[id] = popular # Appending the mode to the market_dict

In [ ]:
market_id_unique = dd.market_id.unique().tolist()

In [ ]:
[market_sub(id) for id in market_id_unique]
market_dict

In [ ]:
# Creating a dataframe with the null values
null_primary_df = dd[dd.store_primary_category.isna()]

# Replacing them
null_primary_df['store_primary_category'] = null_primary_df.market_id.map(market_dict)

In [ ]:
# Dropping null indexes
null_indexes = null_primary_df.index 
dd.drop(index=null_indexes, inplace=True)

# Concatenating 
dd = pd.concat([dd, null_primary_df])

In [ ]:
dd.store_primary_category.isna().sum()

###### All the null values have been replaced.

###### Let's see what are the remaining ones.

In [ ]:
dd.isna().sum()

In [ ]:
dd.order_protocol.value_counts()

In [ ]:
# Creating an empty df
order_p_df = pd.DataFrame()

# Replacing values
for x in unique_id:
    try:
        most_popular = dd[dd.store_id == x]['order_protocol'].mode()[0]
        rep_df = dd[dd.store_id == x]['order_protocol'].fillna(most_popular)
        order_p_df = pd.concat([order_p_df, rep_df])
    except:
        null_protocol = dd[dd.store_id == x]['order_protocol']
        order_p_df = pd.concat([order_p_df, null_protocol])

In [ ]:
order_p_df[0]

In [ ]:
dd['order_protocol'] = order_p_df[0]
dd.order_protocol.isna().sum()

###### Only three null values remains, they could be substituted with the most common order protocol for that food.

In [ ]:
# Finding the remaining stores with null order protocol
null_stores = dd[dd.order_protocol.isna()]['store_id'].unique().tolist()

# Creating an empty df
remaining_null = pd.DataFrame()

# Substituting

for x in null_stores:
    food_category = dd[dd.store_id==x]['store_primary_category'].mode()[0]
    order_protocol_for_food = dd[dd.store_primary_category == food_category]['order_protocol'].mode()[0]
    sub_df = dd[dd.store_id == x]['order_protocol'].fillna(order_protocol_for_food)
    remaining_null = pd.concat([remaining_null, sub_df])

In [ ]:
dd[dd.order_protocol.isna()]

In [ ]:
# Null order protocol 
null_order_protocol = dd[dd.order_protocol.isna()]

# Substituting
null_order_protocol['order_protocol'] = remaining_null[0]

# Null values indexes to drop 
drop_index = dd[dd.order_protocol.isna()].index 

# Dropping
dd.drop(index=drop_index, inplace=True)

# Adding the new values
dd = pd.concat([dd, null_order_protocol])

# Checking the result
dd.order_protocol.isna().sum()

In [ ]:
dd.head()

In [ ]:
dd.isna().sum()

# Total busy dashers

###### Replacing with the average value of the store, else, with the average vvalue of the market.

In [ ]:
def dash_and_orders(column):
    df_to_append = pd.DataFrame()
    try:
        for x in unique_id:
            avg = dd[dd.store_id == x][column].mean()
            df_remove_null = dd[dd.store_id == x][column].fillna(avg)
            df_to_append = pd.concat([df_to_append, df_remove_null])
    except:
        df_with_null = dd[dd.store_id==x][column]
        df_to_append = pd.concat([df_to_append, df_with_null])
    return df_to_append 

In [ ]:
# Total busy dashers
busy_dashers = dash_and_orders('total_busy_dashers')
busy_dashers

In [ ]:
dd['total_busy_dashers'] = busy_dashers[0]
dd.total_busy_dashers.isna().sum()

###### Now, it is possible to use the average of the market id.

In [ ]:
avg_mkt_dict = {}

In [ ]:
def avg_mkt(id):
    avg = round(dd[dd.market_id == id]['total_busy_dashers'].mean())
    avg_mkt_dict[id] = avg

In [ ]:
[avg_mkt(id) for id in market_id_unique]
avg_mkt_dict

In [ ]:
# Creating a dataframe with null values
df_to_add = dd[dd.total_busy_dashers.isna()]

# Finding indexes to drop
indexes_to_remove = dd[dd.total_busy_dashers.isna()].index 

# Substituting values
df_to_add['total_busy_dashers'] = df_to_add.market_id.map(avg_mkt_dict)

# Removing values
dd.drop(index=indexes_to_remove, inplace=True) 

# Adding the new ones
dd = pd.concat([dd, df_to_add])

# Checking the result
dd.total_busy_dashers.isna().sum()

In [ ]:
dd.isna().sum()

# Total onshift dashers

In [ ]:
onshift_dashers = dash_and_orders('total_onshift_dashers')
onshift_dashers

In [ ]:
dd['total_onshift_dashers'] = onshift_dashers[0]

In [ ]:
onshift_dict = {}

In [ ]:
def avg_mkt(id, column):
    avg = round(dd[dd.market_id == id][column].mean())
    onshift_dict[id] = avg

In [ ]:
[avg_mkt(id, 'total_onshift_dashers') for id in market_id_unique]
onshift_dict

In [ ]:
# Creating a dataframe with null values
df_to_add = dd[dd.total_onshift_dashers.isna()]

# Finding indexes to drop
indexes_to_remove = dd[dd.total_onshift_dashers.isna()].index 

# Substituting values
df_to_add['total_onshift_dashers'] = df_to_add.market_id.map(onshift_dict)

# Removing values
dd.drop(index=indexes_to_remove, inplace=True) 

# Adding the new ones
dd = pd.concat([dd, df_to_add])

# Checking the result
dd.total_onshift_dashers.isna().sum()

In [ ]:
dd.isna().sum()

# Total outstanding orders

In [ ]:
outstanding_orders = dash_and_orders('total_outstanding_orders')
outstanding_orders

In [ ]:
dd['total_outstanding_orders'] = outstanding_orders[0]

In [ ]:
outstanding_dict = {}

In [ ]:
def avg_mkt(id, column):
    avg = round(dd[dd.market_id == id][column].mean())
    outstanding_dict[id] = avg

In [ ]:
[avg_mkt(id, 'total_outstanding_orders') for id in market_id_unique]
outstanding_dict

In [ ]:
# Creating a dataframe with null values
df_to_add = dd[dd.total_outstanding_orders.isna()]

# Finding indexes to drop
indexes_to_remove = dd[dd.total_outstanding_orders.isna()].index 

# Substituting values
df_to_add['total_outstanding_orders'] = df_to_add.market_id.map(outstanding_dict)

# Removing values
dd.drop(index=indexes_to_remove, inplace=True) 

# Adding the new ones
dd = pd.concat([dd, df_to_add])

# Checking the result
dd.total_outstanding_orders.isna().sum()

In [ ]:
dd.isna().sum()

# Estimated store to consumer driving duration

In [ ]:
duration = dash_and_orders('estimated_store_to_consumer_driving_duration')
duration

In [ ]:
dd['estimated_store_to_consumer_driving_duration'] = duration[0]

In [ ]:
duration_dict = {}

In [ ]:
def avg_mkt(id, column):
    avg = round(dd[dd.market_id == id][column].mean())
    duration_dict[id] = avg

In [ ]:
[avg_mkt(id, 'estimated_store_to_consumer_driving_duration') for id in market_id_unique]
duration_dict

In [ ]:
# Creating a dataframe with null values
df_to_add = dd[dd.estimated_store_to_consumer_driving_duration.isna()]

# Finding indexes to drop
indexes_to_remove = dd[dd.estimated_store_to_consumer_driving_duration.isna()].index 

# Substituting values
df_to_add['estimated_store_to_consumer_driving_duration'] = df_to_add.market_id.map(duration_dict)

# Removing values
dd.drop(index=indexes_to_remove, inplace=True) 

# Adding the new ones
dd = pd.concat([dd, df_to_add])

# Checking the result
dd.estimated_store_to_consumer_driving_duration.isna().sum()

In [ ]:
dd.isna().sum()

###### All the null values have been removed.

###### Saving the new dataframe in a separate csv file.

In [ ]:
dd.to_csv('dd_no_null.csv', index=None)

# Defining Inputs and Target

In [ ]:
dd = pd.read_csv('dd_no_null.csv', parse_dates=['created_at'])

In [ ]:
dd.info()

# Some columns can be dropped:

* actual_delivery_time
* store_id
* store_primary_category
     

In [ ]:
dd['store_id']

In [ ]:
dd['actual_delivery_time']

In [ ]:
dd.info()

In [ ]:
X = dd.drop(columns=['actual_delivery_time', 'store_id', 'Target'])
y = dd.Target

In [ ]:
X.info()

# Feature Engineering on X

## New columns related to the creation's date

In [ ]:
X.head()

In [ ]:
X['year'] = X.created_at.dt.year 
X['month'] = X.created_at.dt.month 
X['day_of_week'] = X.created_at.dt.day
X['hour'] = X.created_at.dt.hour 
X['minutes'] = X.created_at.dt.minute  
X['day_of_year'] = X.created_at.dt.strftime('%Y-%U')

###### Now it is possible to drop the 'created_at' feature.

In [ ]:
X.drop(columns='created_at', inplace=True)
X.head()

# Other ideas for feature engineering
* price per item: subtotal/total_items
- busy dashers over total dashers:        
- total_busy_dashers/total_onshift_dashers
- price range: max_item_price-min_item_price
- number of dashers per outstanding order: (total_onshift_dashers)/total_outstanding_orders
- price range / (total_items)^2 = (max_item_price - min_item_price) / np.square(total_items)
- order + drive = estimated_order_place_duration + estimated_store_to_consumer_driving_duration

# Price per item

In [ ]:
X['price_per_item'] = X.subtotal/X.total_items

# Number of dashers per outstanding order

In [ ]:
X['dashers_order_ratio'] = X.total_onshift_dashers/(X.total_outstanding_orders+7)

# Price range on items

In [ ]:
X['price_on_items'] = (X.max_item_price - X.min_item_price) / np.square(X.total_items)

# Order + drive

In [ ]:
X['order_and_drive'] = X.estimated_order_place_duration + X.estimated_store_to_consumer_driving_duration

# Selecting Categorical Columns to Encode

In [ ]:
!pip install -U scikit-learn

In [89]:
# from sklearn.preprocessing import OneHotEncoder

categorical_cols = ['market_id', 'order_protocol', 'store_primary_category'] 

# encoder = OneHotEncoder()

# X[encoded_cols] = encoder.fit_transform(X[categorical_cols]) 

# encoded_cols = encoder.get_feature_names(categorical_cols)